In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch.neuron

# BERTモデル及び、tokenizerのの読み込み
bert_path = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(bert_path)
tokenizer2 = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained(bert_path, return_dict=False)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# コンパイル時に使用するダミーtext。モデルに合わせたtokenizer.mask_tokenを使用すること
sample_input = "これはtrace時[MASK]"
sample_input2 = "使うためのダミーテキストですねこれは激熱"

# 入力の最大系列長を事前に指定する
# inputs['input_ids'].size() -> torch.Size([1, 256])
# に統一している。もっと長い入力でコンパイルをかけたい場合は、max_lengthを変更して
max_length = 128
inputs = tokenizer.encode_plus(
   sample_input,
   sample_input2,
   return_tensors="pt",
   padding="max_length",
   max_length=max_length,
   truncation=True
)
tuple_inputs = inputs["input_ids"], inputs["attention_mask"], inputs["token_type_ids"]
print("input_ids:", inputs["input_ids"].size())
print("attention_mask:", inputs["attention_mask"].size())

# コンパイル (Neuron)
model_neuron = torch.neuron.trace(model, tuple_inputs)
# torch.jitの場合 (CPU)
# traced_model = torch.jit.trace(model, example_input, strict=False)

# neuton実行用モデルの保存
# model_neuron.save("bert_neuron.pt")



input_ids: torch.Size([1, 128])
attention_mask: torch.Size([1, 128])


INFO:Neuron:There are 3 ops of 1 different types in the TorchScript that are not compiled by neuron-cc: aten::embedding, (For more information see https://github.com/aws/aws-neuron-sdk/blob/master/release-notes/neuron-cc-ops/neuron-cc-ops-pytorch.md)
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 563, fused = 546, percent fused = 96.98%
INFO:Neuron:Number of neuron graph operations 1596 did not match traced graph 1308 - using heuristic matching of hierarchical information
INFO:Neuron:Compiling function _NeuronGraph$2092 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ubuntu/pytorch_venv/bin/neuron-cc compile /tmp/tmp7hklhmbz/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmp7hklhmbz/graph_def.neff --io-config {"inputs": {"0:0": [[1, 128, 768], "float32"], "1:0": [[1, 1, 1, 128], "float32"], "2:0": [[32000, 768], "float32"]}, "outputs": ["BertOnlyMLMHead_11/BertLMPredictionHead_1/Linear_4/aten_linear/Add:0"]} --verbos

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
.....
Compiler status PASS


INFO:Neuron:skip_inference_context for tensorboard symbols at /home/ubuntu/pytorch_venv/lib/python3.7/site-packages/torch_neuron/tensorboard.py:305 tb_parse
INFO:Neuron:Number of neuron graph operations 1596 did not match traced graph 1308 - using heuristic matching of hierarchical information
INFO:Neuron:Number of arithmetic operators (post-compilation) before = 563, compiled = 546, percent compiled = 96.98%
INFO:Neuron:The neuron partitioner created 1 sub-graphs
INFO:Neuron:Neuron successfully compiled 1 sub-graphs, Total fused subgraphs = 1, Percent of model sub-graphs successfully compiled = 100.0%
INFO:Neuron:Compiled these operators (and operator counts) to Neuron:
INFO:Neuron: => aten::Int: 96
INFO:Neuron: => aten::add: 36
INFO:Neuron: => aten::contiguous: 12
INFO:Neuron: => aten::div: 12
INFO:Neuron: => aten::dropout: 37
INFO:Neuron: => aten::gelu: 13
INFO:Neuron: => aten::layer_norm: 26
INFO:Neuron: => aten::linear: 74
INFO:Neuron: => aten::matmul: 24
INFO:Neuron: => aten::per

In [12]:
#実験
outputs = model_neuron(*tuple_inputs)
print(outputs)

(tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]]),)
